In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [29]:
torch.manual_seed(1)

In [30]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [31]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [32]:
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
hypothesis = 1/(1 + torch.exp(-(x_train.matmul(W) + b))) # 1 / (1 + exp(-X)), x = x_train @ W + b
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


In [33]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)

In [34]:
losses = -(y_train * torch.log(hypothesis) + (1-y_train) * torch.log(1-hypothesis))
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


In [35]:
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

In [36]:
# Full code
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs+1):

    hypo = torch.sigmoid(x_train.matmul(W) + b)
    cost = F.binary_cross_entropy(hypo, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch%100==0:
        print(f'Ep {epoch}/{nb_epochs} | Cost {cost:.5f}')

Ep 0/1000 | Cost 0.69315
Ep 100/1000 | Cost 0.13472
Ep 200/1000 | Cost 0.08064
Ep 300/1000 | Cost 0.05790
Ep 400/1000 | Cost 0.04530
Ep 500/1000 | Cost 0.03726
Ep 600/1000 | Cost 0.03167
Ep 700/1000 | Cost 0.02756
Ep 800/1000 | Cost 0.02439
Ep 900/1000 | Cost 0.02189
Ep 1000/1000 | Cost 0.01985


In [37]:
hypo_last = torch.sigmoid(x_train.matmul(W)+b)
print(hypo_last)

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01],
        [9.9969e-01]], grad_fn=<SigmoidBackward0>)


In [38]:
prediction = hypo_last >= torch.FloatTensor([0.5])
print(prediction.byte())

tensor([[0],
        [0],
        [0],
        [1],
        [1],
        [1]], dtype=torch.uint8)


In [39]:
# 좀더 pytorch스럽게
class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.linear = nn.Linear(8, 1) # 8개 input, 1개의 output. W와 b가 모두 들어있는 linear layer이다.
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

model = BinaryClassifier()

In [42]:
# 예시 데이터를 나중에 만들어서 사용해 보기로... (torch.randn 등등)
# x_train.shape = m * 8
# y_train.shape = m * 1
# m은 마음대로.
x_train = torch.rand(20, 8).float()
y_train = torch.randint(2, (20, 1)).float()
print(x_train)
print(y_train)

tensor([[0.4388, 0.6387, 0.5247, 0.6826, 0.3051, 0.4635, 0.4550, 0.5725],
        [0.4980, 0.9371, 0.6556, 0.3138, 0.1980, 0.4162, 0.2843, 0.3398],
        [0.5239, 0.7981, 0.7718, 0.0112, 0.8100, 0.6397, 0.9743, 0.8300],
        [0.0444, 0.0246, 0.2588, 0.9391, 0.4167, 0.7140, 0.2676, 0.9906],
        [0.2885, 0.8750, 0.5059, 0.2366, 0.7570, 0.2346, 0.6471, 0.3556],
        [0.4452, 0.0193, 0.2616, 0.7713, 0.3785, 0.9980, 0.9008, 0.4766],
        [0.1663, 0.8045, 0.6552, 0.1768, 0.8248, 0.8036, 0.9434, 0.2197],
        [0.4177, 0.4903, 0.5730, 0.1205, 0.1452, 0.7720, 0.3828, 0.7442],
        [0.5285, 0.6642, 0.6099, 0.6818, 0.7479, 0.0369, 0.7517, 0.1484],
        [0.1227, 0.5304, 0.4148, 0.7937, 0.2104, 0.0555, 0.8639, 0.4259],
        [0.7812, 0.6607, 0.1251, 0.6004, 0.6201, 0.1652, 0.2628, 0.6705],
        [0.5896, 0.2873, 0.3486, 0.9579, 0.4075, 0.7819, 0.7165, 0.1768],
        [0.0748, 0.9799, 0.5261, 0.8427, 0.6036, 0.6608, 0.8735, 0.9741],
        [0.1682, 0.5625, 0.8731, 0.862

In [43]:
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

    hypothesis = model(x_train) # Linear -> Sigmoid
    cost = F.binary_cross_entropy(hypothesis, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Ep:{epoch}/{nb_epochs} | Cost:{cost:.6f}")

Ep:0/100 | Cost:0.723373
Ep:10/100 | Cost:0.583882
Ep:20/100 | Cost:0.559285
Ep:30/100 | Cost:0.540712
Ep:40/100 | Cost:0.526272
Ep:50/100 | Cost:0.514721
Ep:60/100 | Cost:0.505232
Ep:70/100 | Cost:0.497249
Ep:80/100 | Cost:0.490390
Ep:90/100 | Cost:0.484388
Ep:100/100 | Cost:0.479053
